In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import astropy.units as u

import pysynphot as sp

In [2]:
feh_lmc = -0.5
feh_smc = -1.0

ebv_lmc = 0.13 # Massey et al. 2007
ebv_smc = 0.09

lmc_ext = sp.reddening.Extinction(ebv_lmc,'lmcavg') #Gordon et al. (2003, ApJ, 594, 279) R_V = 3.41)
smc_ext = sp.reddening.Extinction(ebv_smc,'smcbar') #R_V=2.74


#Values for B stars from Silaj et al. 2014, ApJ, 795:82 https://iopscience.iop.org/article/10.1088/0004-637X/795/1/82/pdf
B0_Teff = 2.5e4
B0_logg = 4.0
B0_R = (10.0 * u.Rsun).to(u.pc)
B9_Teff = 1.06e4
B9_logg = 4.1
B9_R = (2.7 * u.Rsun).to(u.pc)

In [3]:
#Some bookkeeping: we're going to load in the TESS bandpass, convolve with Vega to derive the zero-point
vega = sp.Vega

T_df = pd.read_csv('../data/tess-response-function-v2.0.csv',skiprows=7,names=['Wavelength','Transmission'])
T_bp = sp.ArrayBandpass(T_df['Wavelength'].values * 10.0, T_df['Transmission'], name='TESS')

vega_obs = sp.Observation(vega, T_bp)
vega_zp = vega_obs.effstim('obmag')
vega_zp

(TESS) does not have a defined binset in the wavecat table. The waveset of the spectrum will be used instead.


-27.118853442069746

In [4]:
for gal,met,dm,ext in zip(['LMC','SMC'],[feh_lmc,feh_smc],[18.52,19.05],[lmc_ext,smc_ext]): #distance moduli from Kovacs 2000a,b
    
    dist = np.power(10.0, 1+(dm / 5.0)) * u.pc
    
    B0_cat = sp.Icat('k93models', B0_Teff, met, B0_logg) #Kurucz 1993 models
    B0_flux = B0_cat.flux * (B0_R/dist)**2.0
    B0 = sp.ArraySpectrum(B0_cat.wave,B0_flux,fluxunits='flam')
    B0_red = B0*ext
    B0_obs = sp.Observation(B0_red, T_bp)
    B0_mag = B0_obs.effstim('vegamag') #- vega_zp
    
    B9_cat = sp.Icat('k93models', B9_Teff, met, B9_logg) #Kurucz 1993 models
    B9_flux = B9_cat.flux * (B9_R/dist)**2.0
    B9 = sp.ArraySpectrum(B9_cat.wave,B9_flux,fluxunits='flam')
    B9_red = B9*ext
    B9_obs = sp.Observation(B9_red, T_bp)
    B9_mag = B9_obs.effstim('vegamag') #- vega_zp
    
    print(f'Main sequence B stars in the {gal} have typical TESS magnitudes of {B0_mag}-{B9_mag}')

(TESS) does not have a defined binset in the wavecat table. The waveset of the spectrum will be used instead.
(TESS) does not have a defined binset in the wavecat table. The waveset of the spectrum will be used instead.
Main sequence B stars in the LMC have typical TESS magnitudes of 14.965109540878695-19.233247338692518
(TESS) does not have a defined binset in the wavecat table. The waveset of the spectrum will be used instead.
(TESS) does not have a defined binset in the wavecat table. The waveset of the spectrum will be used instead.
Main sequence B stars in the SMC have typical TESS magnitudes of 15.350051026668217-19.60559564527261
